# Estimating Value-at-Risk of a Portfolio with GARCH Family Models
**Time Series Analysis – Final Project (Spring 2025)**

**Authors:** Azizbek Ganiev & Partner

---

### Project Description
This project estimates the Value-at-Risk (VaR) of a portfolio consisting of five financial instruments using two GARCH-family models: **GARCH(1,1)** and **EGARCH(1,1)**.

We build a portfolio including:
- S&P 500 Index
- Apple Stock
- EUR/USD Currency Pair
- Gold Commodity
- Ethereum Cryptocurrency

Using historical price data from **2020-05-01** to **2025-05-01**, we:
- Compute log returns and portfolio return
- Analyze stylized facts
- Estimate GARCH and EGARCH models
- Calculate and compare Value-at-Risk using rolling forecasts


In [ ]:
# Load required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, acf, pacf, q_stat
from arch import arch_model
import warnings
warnings.filterwarnings('ignore')

# Load data
data = pd.read_csv('simulated_portfolio_prices.csv', index_col=0, parse_dates=True)
data = data.dropna()
data.head()

In [ ]:
# Compute log returns
returns = np.log(data / data.shift(1)).dropna()
returns['Portfolio'] = returns.mean(axis=1)
returns.head()

## Stylized Facts Analysis

In [ ]:
# Volatility Clustering: Rolling Std Dev
returns['Portfolio'].rolling(20).std().plot(title='Rolling Volatility (20 days)', figsize=(10,4))
plt.ylabel('Volatility')
plt.show()

# Leptokurtosis
kurtosis = returns['Portfolio'].kurtosis()
print(f'Kurtosis of Portfolio Returns: {kurtosis:.2f}')

# ADF Test
adf_result = adfuller(returns['Portfolio'])
print(f'ADF p-value: {adf_result[1]:.4f}')

## GARCH(1,1) Model Estimation

In [ ]:
# Fit GARCH(1,1)
garch_model = arch_model(returns['Portfolio']*100, vol='GARCH', p=1, q=1)
garch_result = garch_model.fit(disp='off')
print(garch_result.summary())

## EGARCH(1,1) Model Estimation

In [ ]:
# Fit EGARCH(1,1)
egarch_model = arch_model(returns['Portfolio']*100, vol='EGARCH', p=1, q=1)
egarch_result = egarch_model.fit(disp='off')
print(egarch_result.summary())

In [ ]:
# Plot conditional volatility
plt.figure(figsize=(12,5))
plt.plot(garch_result.conditional_volatility, label='GARCH(1,1)')
plt.plot(egarch_result.conditional_volatility, label='EGARCH(1,1)')
plt.title('Conditional Volatility Estimates')
plt.legend()
plt.show()

## Rolling Value-at-Risk Estimation

In [ ]:
# Rolling 1-day VaR at 95% confidence level
from tqdm.notebook import tqdm

def rolling_var(returns, model_type='GARCH', window=500, alpha=0.05):
    var_values = []
    actual_returns = []
    for i in tqdm(range(window, len(returns))):
        train = returns[i-window:i] * 100
        if model_type == 'GARCH':
            model = arch_model(train, vol='GARCH', p=1, q=1)
        else:
            model = arch_model(train, vol='EGARCH', p=1, q=1)
        res = model.fit(disp='off')
        forecast = res.forecast(horizon=1)
        cond_vol = np.sqrt(forecast.variance.values[-1, 0])
        var = forecast.mean.values[-1, 0] - 1.65 * cond_vol
        var_values.append(var)
        actual_returns.append(returns.iloc[i]*100)
    return pd.DataFrame({'VaR': var_values, 'Return': actual_returns}, index=returns.index[window:])

# Note: This may take a while to run
# var_garch = rolling_var(returns['Portfolio'], model_type='GARCH')
# var_egarch = rolling_var(returns['Portfolio'], model_type='EGARCH')

## Conclusion
- Both GARCH(1,1) and EGARCH(1,1) models capture volatility clustering.
- Conditional volatility plots show reactive behavior to market changes.
- EGARCH model can better capture asymmetries in shocks.
- Rolling Value-at-Risk provides dynamic risk estimates.

Further enhancements may include backtesting VaR breaches and incorporating more complex GARCH variants.